In [1]:
def dhcp_endpointstocortex_pipeline(subject, session, local_env=False):
    """
    For each `subject`, `session` pair run tractography pipeline.
    systemctl --user start docker-desktop 
    0. Perpare local environment
    1. Generate isotropic DWI files
    2. Realign DWI to anatomy using rigid transformation
    3. Estimate CSD response function
    4. Reassign ribbon values
    5. Generate five-tissue-type (5TT) segmented tissue image
    6. Tractography
    7. Create BIDS derivatives
    8. Upload to s3
    
    Parameters
    ----------
    subject : string
    session : string
    aws_access_key : string
    aws_secret_key : string
    streamline_count : int
    local_env : boolean
    """
    
    import subprocess
    import os
    from os.path import exists, join
    import s3fs
    import json
    from AFQ.definitions.mapping import AffMap
    import nibabel as nib
    import numpy as np
    import os.path as op
    import re
    import pandas as pd
    import nipype.interfaces.freesurfer
      
    fs = s3fs.S3FileSystem()

    def print_quiet(string):
        """
        mrtrix doesn't provide a way to conviently supress progress messages without
        also supressing informational messages. 
        
        filter stderr/stdout before printing

        """
        import re
        
        # remove any line that looks like a progress message
        string = re.sub(r'.*\[[0-9 ]{3}\%\].*\n?', '', string)

        # remove empty lines
        string = re.sub(r'^\n$', '', string)
        
        # remove any leading or trailing whitespace
        string = string.strip()
        
        if string != "":
            print(string, flush=True)

    ###############################################################################
    # Step 0: Prepare local environment and downlaod data
    ###############################################################################
    cmd='export SUBJECTS_DIR=/output'
    print(cmd)
    out = os.system(cmd)
    
    print('Step 0: Prepare local environment', flush=True)
    
    os.makedirs(join('input', f'sub-{subject}', f'ses-{session}'), exist_ok=True)
    os.makedirs(join('output', f'sub-{subject}', f'ses-{session}'), exist_ok=True)
    os.makedirs(join('output', f'sub-{subject}', f'ses-{session}','tracts'), exist_ok=True)   
    os.makedirs(join('output', f'sub-{subject}', f'ses-{session}','volume'), exist_ok=True)
    os.makedirs(join('output', f'sub-{subject}', f'ses-{session}','label'), exist_ok=True)
    os.makedirs(join('output', f'sub-{subject}', f'ses-{session}','t1t2values'), exist_ok=True)
    os.makedirs(join('output', f'sub-{subject}', f'ses-{session}','label', 'annotation'), exist_ok=True)
    os.makedirs(join('output', f'sub-{subject}', f'ses-{session}','label', 'ROIs'), exist_ok=True)
    os.makedirs(join('output', f'sub-{subject}', f'ses-{session}','label', 'FinalLabels'), exist_ok=True)

    
    bundles = ['ARCL', 'ARCR', 'ATRL', 'ATRR', 'CGCL', 'CGCR', 'CSTL', 'CSTR', 'FA', 'FP', 'IFOL', 'IFOR', 'ILFL', 'ILFR', 
                  'MdLFL', 'MdLFR', 'ORL', 'ORR', 'pARCL', 'pARCR', 'SLFL', 'SLFR', 'UNCL', 'UNCR', 'VOFL', 'VOFR']    
    rightBundles = ['ARCR', 'ATRR', 'CGCR', 'CSTR', 'FA', 'FP', 'IFOR', 'ILFR', 'MdLFR', 'ORR', 'pARCR', 'SLFR', 'UNCR', 'VOFR']
    leftBundles = ['ARCL', 'ATRL', 'CGCL', 'CSTL', 'FA', 'FP', 'IFOL', 'ILFL', 'MdLFL', 'ORL', 'pARCL', 'SLFL', 'UNCL', 'VOFL']


    ###############################################################################
    # Step 1: Convert tract files from .trk to .tck
    ###############################################################################

    for bundle in bundles:
        trk2tck = subprocess.run(
            [
                'nib-trk2tck',
                f'output/sub-{subject}/ses-{session}/tracts/sub-{subject}_ses-{session}_coordsys-RASMM_trkmethod-probCSD_recogmethod-AFQ_desc-{bundle}_tractography.trk',
            ],
            check = True,
            capture_output = True,
            text=True
        )
        print_quiet(trk2tck.stdout)
        print_quiet(trk2tck.stderr)

    ###############################################################################
    # Step 2: Use tckmap to find endpoints in volume space
    ###############################################################################
    
    for bundle in bundles:
        tckmap = subprocess.run(
            [
                'tckmap',
                '-template',
                f'input/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_T1w.nii.gz',
                '-ends_only',
                '-info',
                '-contrast',
                'tdi',
                '-force',
                f'output/sub-{subject}/ses-{session}/tracts/sub-{subject}_ses-{session}_coordsys-RASMM_trkmethod-probCSD_recogmethod-AFQ_desc-{bundle}_tractography.tck',
                f'output/sub-{subject}/ses-{session}/volume/sub-{subject}_ses-{session}_tckmap_bundle-{bundle}.nii.gz'           
            ],
            check = True,
            capture_output = True,
            text=True
        )
        print_quiet(tckmap.stdout)
        print_quiet(tckmap.stderr)

    ###############################################################################
    # Step 3: Double endpoint-niftii files from uni32 to uni64
    ###############################################################################
    
    for bundle in bundles:
        nii = nib.load(os.path.join(f"output/sub-{subject}/ses-{session}/volume/", f'sub-{subject}_ses-{session}_tckmap_bundle-{bundle}.nii.gz')) 
        niidata = nii.get_fdata()
        niidouble = np.double(niidata)
        niidouble_img = nib.Nifti1Image(niidouble, nii.affine)
        nib.save(niidouble_img,f'output/sub-{subject}/ses-{session}/volume/sub-{subject}_ses-{session}_tckmapUNI64_bundle-{bundle}.nii.gz')
        
    ###############################################################################
    # Step 4: Function to use mrivol2surf without freesurfer preprocessing (2nd release)
    ###############################################################################

    def aligndHCPFreeSurferSurf(ref_volume, trg_surface, work_dir):
        ### create working volume directory
        mri_dir = op.join(work_dir, "mri")
        os.makedirs(mri_dir, exist_ok = True)

        ### convert reference volume into freesurfer mgz
        nib.save(nib.load(ref_volume), op.join(mri_dir, "ref.mgz"))
        ref_volume = nib.load(op.join(mri_dir, "ref.mgz"))
        xyz = ref_volume.header.get("Pxyz_c") # volume center

        ### create working surface directory
        surf_dir = op.join(work_dir, "surf")
        os.makedirs(surf_dir, exist_ok = True)

        for trg_fname in trg_surface: # for each surface file
            ### load coordinates and faces of surface file
            coords, faces = nib.load(trg_fname).agg_data()

            ### center surface vertices
            coords = np.apply_along_axis(lambda x: coords - x, -1, xyz)   

            ### rotate surface vertices
            theta = np.deg2rad(-90) # rotation degrees 
            R = np.array( # rotation matrix
                [[1, 0, 0], 
                 [0, np.cos(theta), -np.sin(theta)],
                 [0, np.sin(theta),  np.cos(theta)]]
            )
            coords = np.matmul(R, coords.T).T

            ### save coregistered surface file
            hemi_str = "lh" if "hemi-left" in trg_fname else "rh"
            suffix = re.sub("\..+", "", trg_fname.split("_")[-1])
            nib.freesurfer.io.write_geometry(
                op.join(surf_dir, f"{hemi_str}.{suffix}"), coords, faces)
            


    ###############################################################################
    # Step 5: Running mrivol2surf on left hemisphere
    ###############################################################################

    proj_values=np.arange(-3,3.5,0.5)
    ref_volume=f'input/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_T1w.nii.gz'
    trg_surface=[f'input/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_hemi-left_wm.surf.gii', f'input/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_hemi-right_wm.surf.gii'] 
    work_dir=f'output/sub-{subject}/ses-{session}/'

    for bundle in leftBundles:
        for p in proj_values:
            main(ref_volume, trg_surface, work_dir)
            mrivol2surf = subprocess.run(
                [
                    'mri_vol2surf',
                    '--sd',
                    'output/',
                    '--o',
                    f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_mrivol2surf_bundle-{bundle}_lh_surface_{p}.mgh',
                    '--regheader',
                    f'sub-{subject}/ses-{session}',
                    '--hemi',
                    'lh',
                    '--surf',
                    'wm',
                    '--mov',
                    f'output/sub-{subject}/ses-{session}/volume/sub-{subject}_ses-{session}_tckmapUNI64_bundle-{bundle}.nii.gz',
                    '--ref',
                    'ref.mgz',
                    '--projdist',
                    f'{p}',
                    '--fwhm',
                    '1'         
                ],
                #check = True,
                capture_output = True,
                text=True
                )
            print(mrivol2surf.stdout)
            print(mrivol2surf.stderr)

    ###############################################################################
    # Step 6: Running mrivol2surf on right hemisphere
    ###############################################################################

    for bundle in rightBundles:
        for p in proj_values:
            main(ref_volume, trg_surface, work_dir)
            mrivol2surf = subprocess.run(
                [
                    'mri_vol2surf',
                    '--sd',
                    'output/',
                    '--o',
                    f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_mrivol2surf_bundle-{bundle}_rh_surface_{p}.mgh',
                    '--regheader',
                    f'sub-{subject}/ses-{session}',
                    '--hemi',
                    'rh',
                    '--surf',
                    'wm',
                    '--mov',
                    f'output/sub-{subject}/ses-{session}/volume/sub-{subject}_ses-{session}_tckmapUNI64_bundle-{bundle}.nii.gz',
                    '--ref',
                    'ref.mgz',
                    '--projdist',
                    f'{p}',
                    '--fwhm',
                    '1'         
                ],
                #check = True,
                capture_output = True,
                text=True
                )
            print_quiet(mrivol2surf.stdout)
            print_quiet(mrivol2surf.stderr)

    ###############################################################################
    # Step 7: Running mriconcat on left hemisphere to find maximum endpoints
    ###############################################################################
    
    for bundle in leftBundles:
        filepath = f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_bundle-{bundle}_lh_proj_max.mgh'
        if os.path.exists(filepath):
            os.remove(filepath)
        else:
            print("Can not delete the file as it doesn't exist")
            
        mriconcat = subprocess.run(
            [
                'mri_concat',
                '--i',
                f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_mrivol2surf_bundle-{bundle}_lh_surface_-3.0.mgh',
                f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_mrivol2surf_bundle-{bundle}_lh_surface_-2.5.mgh',
                f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_mrivol2surf_bundle-{bundle}_lh_surface_-2.0.mgh',
                f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_mrivol2surf_bundle-{bundle}_lh_surface_-1.5.mgh',
                f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_mrivol2surf_bundle-{bundle}_lh_surface_-1.0.mgh',
                f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_mrivol2surf_bundle-{bundle}_lh_surface_-0.5.mgh',
                f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_mrivol2surf_bundle-{bundle}_lh_surface_0.0.mgh',
                f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_mrivol2surf_bundle-{bundle}_lh_surface_0.5.mgh',
                f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_mrivol2surf_bundle-{bundle}_lh_surface_1.0.mgh',
                f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_mrivol2surf_bundle-{bundle}_lh_surface_1.5.mgh',
                f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_mrivol2surf_bundle-{bundle}_lh_surface_2.0.mgh',
                f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_mrivol2surf_bundle-{bundle}_lh_surface_2.5.mgh',
                f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_mrivol2surf_bundle-{bundle}_lh_surface_3.0.mgh',                                                                                                                      
                '--o',
                f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_bundle-{bundle}_lh_proj_max.mgh',
                '--max'
            ],
            check=True,
            capture_output=True,
            text=True
            )
        
        for p in proj_values:
            os.remove(f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_mrivol2surf_bundle-{bundle}_lh_surface_{p}.mgh')

    ###############################################################################
    # Step 8: Running mriconcat on right hemisphere to find maximum endpoints
    ###############################################################################
    
    for bundle in rightBundles:
        filepath = f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_bundle-{bundle}_rh_proj_max.mgh'
        if os.path.exists(filepath):
            os.remove(filepath)
        else:
            print("Can not delete the file as it doesn't exist")
    
        mriconcat = subprocess.run(
            [
                'mri_concat',
                '--i',
                f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_mrivol2surf_bundle-{bundle}_rh_surface_-3.0.mgh',
                f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_mrivol2surf_bundle-{bundle}_rh_surface_-2.5.mgh',
                f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_mrivol2surf_bundle-{bundle}_rh_surface_-2.0.mgh',
                f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_mrivol2surf_bundle-{bundle}_rh_surface_-1.5.mgh',
                f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_mrivol2surf_bundle-{bundle}_rh_surface_-1.0.mgh',
                f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_mrivol2surf_bundle-{bundle}_rh_surface_-0.5.mgh',
                f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_mrivol2surf_bundle-{bundle}_rh_surface_0.0.mgh',
                f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_mrivol2surf_bundle-{bundle}_rh_surface_0.5.mgh',
                f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_mrivol2surf_bundle-{bundle}_rh_surface_1.0.mgh',
                f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_mrivol2surf_bundle-{bundle}_rh_surface_1.5.mgh',
                f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_mrivol2surf_bundle-{bundle}_rh_surface_2.0.mgh',
                f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_mrivol2surf_bundle-{bundle}_rh_surface_2.5.mgh',
                f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_mrivol2surf_bundle-{bundle}_rh_surface_3.0.mgh',
                '--o',
                f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_bundle-{bundle}_rh_proj_max.mgh',
                '--max'
            ],
            check=True,
            capture_output=True,
            text=True
            )
        print_quiet(mriconcat.stdout)
        print_quiet(mriconcat.stderr)
        
        for p in proj_values:
            os.remove(f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_mrivol2surf_bundle-{bundle}_rh_surface_{p}.mgh')
            
    ###############################################################################
    # Step 9: converting dHCP-surface files to freesurfer surface files left hemisphere
    ###############################################################################

    mrisconvert = subprocess.run(
        [
            'mris_convert',
            f'input/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_hemi-left_wm.surf.gii',
            f'output/sub-{subject}/ses-{session}/surf/lh.white'
        ],
        check=True,
        capture_output=True,
        text=True
        )
    print_quiet(mrisconvert.stdout)
    print_quiet(mrisconvert.stderr)

    ###############################################################################
    # Step 10: converting dHCP-surface files to freesurfer surface files right hemisphere
    ###############################################################################

    mrisconvert = subprocess.run(
        [
            'mris_convert',
            f'input/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_hemi-right_wm.surf.gii',
            f'output/sub-{subject}/ses-{session}/surf/rh.white'
        ],
        check=True,
        capture_output=True,
        text=True
        )
    print_quiet(mrisconvert.stdout)
    print_quiet(mrisconvert.stderr)

    ###############################################################################
    # Step 11: Load in annotation file and split it into different labels for left hemi
    ###############################################################################

    annotation_labelLH = subprocess.run(
        [
            'mri_annotation2label',
            '--subject',
            f'sub-{subject}/ses-{session}',
            '--sd',
            'output',
            '--hemi',
            'lh',
            '--annotation',
            f'output/sub-{subject}/ses-{session}/label/sub-{subject}_ses-{session}_hemi-left_desc-drawem_dseg.label.gii',
            '--outdir',
            f'output/sub-{subject}/ses-{session}/label/annotation'
        ],
        check=True,
        capture_output=True,
        text=True
        )

    print_quiet(annotation_labelLH.stdout)
    print_quiet(annotation_labelLH.stderr)

    ###############################################################################
    # Step 12: Load in annotation file and split it into different labels for right hemi
    ###############################################################################

    annotation_labelRH = subprocess.run(
        [
            'mri_annotation2label',
            '--subject',
            f'sub-{subject}/ses-{session}',
            '--sd',
            'output',
            '--hemi',
            'rh',
            '--annotation',
            f'output/sub-{subject}/ses-{session}/label/sub-{subject}_ses-{session}_hemi-right_desc-drawem_dseg.label.gii',
            '--outdir',
            f'output/sub-{subject}/ses-{session}/label/annotation'
        ],
        check=True,
        capture_output=True,
        text=True
        )
    print_quiet(annotation_labelRH.stdout)
    print_quiet(annotation_labelRH.stderr)

    ###############################################################################
    # Step 13: Function to insert text within .label file
    ###############################################################################

    def prepend_multiple_lines(file_name, list_of_lines):
        """Insert given list of strings as a new lines at the beginning of a file"""
    
        # define name of temporary dummy file
        dummy_file = file_name + '.bak'
        # open given original file in read mode and dummy file in write mode
        with open(file_name, 'r') as read_obj, open(dummy_file, 'w') as write_obj:
            # Iterate over the given list of strings and write them to dummy file as lines
            for line in list_of_lines:
                write_obj.write(line + '\n')
            # Read lines from original file one by one and append them to the dummy file
            for line in read_obj:
                write_obj.write(line)
    
        # remove original file
        os.remove(file_name)
        # Rename dummy file as the original file
        os.rename(dummy_file, file_name)

    ###############################################################################
    # Step 14: Load in labels for left hemisphere and save them in different ROIs
    ###############################################################################

    outfolder = f"output/sub-{subject}/ses-{session}/label/ROIs"
         
    labelTable0 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/lh.???.label', skiprows=[0,1], sep='  ', header=None, engine='python')          
    labelTable0[4] = [x.split(' ')[0] for x in labelTable0[3]]
    labelTable0[5] = [x.split(' ')[-1] for x in labelTable0[3]]
    labelTable0 = labelTable0.drop(columns=3)
        
    labelTable1 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/lh.L_Anterior_temporal_lobe_medial.label', skiprows=[0,1], sep='  ', header=None, engine='python')
    labelTable1[4] = [x.split(' ')[0] for x in labelTable1[3]]
    labelTable1[5] = [x.split(' ')[-1] for x in labelTable1[3]]
    labelTable1 = labelTable1.drop(columns=3)
        
    labelTable2 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/lh.L_Anterior_temporal_lobe_lateral.label', skiprows=[0,1], sep='  ', header=None, engine='python')
    labelTable2[4] = [x.split(' ')[0] for x in labelTable2[3]]
    labelTable2[5] = [x.split(' ')[-1] for x in labelTable2[3]]
    labelTable2 = labelTable2.drop(columns=3)
        
    labelTable3 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/lh.L_Gyri_parahippocampalis_et_ambiens_anterior.label', skiprows=[0,1], sep='  ', header=None, engine='python')
    labelTable3[4] = [x.split(' ')[0] for x in labelTable3[3]]
    labelTable3[5] = [x.split(' ')[-1] for x in labelTable3[3]]
    labelTable3 = labelTable3.drop(columns=3)
        
    labelTable4 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/lh.L_Superior_temporal_gyrus_middle.label', skiprows=[0,1], sep='  ', header=None, engine='python')
    labelTable4[4] = [x.split(' ')[0] for x in labelTable4[3]]
    labelTable4[5] = [x.split(' ')[-1] for x in labelTable4[3]]
    labelTable4 = labelTable4.drop(columns=3)
        
    labelTable5 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/lh.L_Medial_and_inferior_temporal_gyri_anterior.label', skiprows=[0,1], sep='  ', header=None, engine='python')
    labelTable5[4] = [x.split(' ')[0] for x in labelTable5[3]]
    labelTable5[5] = [x.split(' ')[-1] for x in labelTable5[3]]
    labelTable5 = labelTable5.drop(columns=3)
        
    labelTable6 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/lh.L_Lateral_occipitotemporal_gyrus_gyrus_fusiformis_anterior.label', skiprows=[0,1], sep='  ', header=None, engine='python')
    labelTable6[4] = [x.split(' ')[0] for x in labelTable6[3]]
    labelTable6[5] = [x.split(' ')[-1] for x in labelTable6[3]]
    labelTable6 = labelTable6.drop(columns=3)
        
    labelTable7 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/lh.L_Insula.label', skiprows=[0,1], sep='  ', header=None, engine='python')
    labelTable7[4] = [x.split(' ')[0] for x in labelTable7[3]]
    labelTable7[5] = [x.split(' ')[-1] for x in labelTable7[3]]
    labelTable7 = labelTable7.drop(columns=3)
        
    labelTable8 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/lh.L_Occipital_lobe.label', skiprows=[0,1], sep='  ', header=None, engine='python')
    labelTable8[4] = [x.split(' ')[0] for x in labelTable8[3]]
    labelTable8[5] = [x.split(' ')[-1] for x in labelTable8[3]]
    labelTable8 = labelTable8.drop(columns=3)
        
    labelTable9 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/lh.L_Gyri_parahippocampalis_et_ambiens_posterior.label', skiprows=[0,1], sep='  ', header=None, engine='python')
    labelTable9[4] = [x.split(' ')[0] for x in labelTable9[3]]
    labelTable9[5] = [x.split(' ')[-1] for x in labelTable9[3]]
    labelTable9 = labelTable9.drop(columns=3)
        
    labelTable10 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/lh.L_Lateral_occipitotemporal_gyrus_gyrus_fusiformis_posterior.label', skiprows=[0,1], sep='  ', header=None, engine='python')
    labelTable10[4] = [x.split(' ')[0] for x in labelTable10[3]]
    labelTable10[5] = [x.split(' ')[-1] for x in labelTable10[3]]
    labelTable10 = labelTable10.drop(columns=3)
        
    labelTable11 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/lh.L_Medial_and_inferior_temporal_gyri_posterior.label', skiprows=[0,1], sep='  ', header=None, engine='python')
    labelTable11[4] = [x.split(' ')[0] for x in labelTable11[3]]
    labelTable11[5] = [x.split(' ')[-1] for x in labelTable11[3]]
    labelTable11 = labelTable11.drop(columns=3)
        
    labelTable12 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/lh.L_Superior_temporal_gyrus_posterior.label', skiprows=[0,1], sep='  ', header=None, engine='python')
    labelTable12[4] = [x.split(' ')[0] for x in labelTable12[3]]
    labelTable12[5] = [x.split(' ')[-1] for x in labelTable12[3]]
    labelTable12 = labelTable12.drop(columns=3)
        
    labelTable13 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/lh.L_Cingulate_gyrus_anterior.label', skiprows=[0,1], sep='  ', header=None, engine='python')
    labelTable13[4] = [x.split(' ')[0] for x in labelTable13[3]]
    labelTable13[5] = [x.split(' ')[-1] for x in labelTable13[3]]
    labelTable13 = labelTable13.drop(columns=3)
        
    labelTable14 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/lh.L_Cingulate_gyrus_posterior.label', skiprows=[0,1], sep='  ', header=None, engine='python')
    labelTable14[4] = [x.split(' ')[0] for x in labelTable14[3]]
    labelTable14[5] = [x.split(' ')[-1] for x in labelTable14[3]]
    labelTable14 = labelTable14.drop(columns=3)
        
    labelTable15 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/lh.L_Frontal_lobe.label', skiprows=[0,1], sep='  ', header=None, engine='python')
    labelTable15[4] = [x.split(' ')[0] for x in labelTable15[3]]
    labelTable15[5] = [x.split(' ')[-1] for x in labelTable15[3]]
    labelTable15 = labelTable15.drop(columns=3)
        
    labelTable16 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/lh.L_Parietal_lobe.label', skiprows=[0,1], sep='  ', header=None, engine='python')
    labelTable16[4] = [x.split(' ')[0] for x in labelTable16[3]]
    labelTable16[5] = [x.split(' ')[-1] for x in labelTable16[3]]
    labelTable16 = labelTable16.drop(columns=3)
        
    #ARC
    labelTable15.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_LH_ARCL_01.label', header=None, index=None, sep=' ', mode='a')

    LabelTableDfs_ARC_L = [labelTable2, labelTable4, labelTable5, labelTable6, labelTable10, labelTable11]
    LabelTable_ARC_L_02 = pd.concat(LabelTableDfs_ARC_L)
    LabelTable_ARC_L_02.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_LH_ARCL_02.label', header=None, index=None, sep=' ', mode='a')
            
    #ATR
    labelTable15.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_LH_ATRL_01.label', header=None, index=None, sep=' ', mode='a')
                
    #CGC
    LabelTableDfs_CGC_L = [labelTable13, labelTable15]
    LabelTable_CGC_L_01 = pd.concat(LabelTableDfs_CGC_L)
    LabelTable_CGC_L_01.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_LH_CGCL_01.label', header=None, index=None, sep=' ', mode='a')
            
    LabelTableDfs_CGC_L_02 = [labelTable9, labelTable14, labelTable16]
    LabelTable_CGC_L_02 = pd.concat(LabelTableDfs_CGC_L_02)
    LabelTable_CGC_L_02.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_LH_CGCL_02.label', header=None, index=None, sep=' ', mode='a')
            
    #CST
    LabelTableDfs_CST_L = [labelTable12, labelTable15, labelTable16]
    LabelTable_CST_L_01 = pd.concat(LabelTableDfs_CST_L)
    LabelTable_CST_L_01.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_LH_CSTL_01.label', header=None, index=None, sep=' ', mode='a')
                        
    #FA
    labelTable15.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_LH_FA_01.label', header=None, index=None, sep=' ', mode='a')
                     
    #FP
    LabelTableDfs_FP_L = [labelTable8, labelTable16]
    LabelTable_FP_L_01 = pd.concat(LabelTableDfs_FP_L)
    LabelTable_FP_L_01.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_LH_FP_01.label', header=None, index=None, sep=' ', mode='a')
                        
    #IFO
    labelTable15.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_LH_IFOL_01.label', header=None, index=None, sep=' ', mode='a')
          
    labelTable8.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_LH_IFOL_02.label', header=None, index=None, sep=' ', mode='a')
            
    #ILF
    LabelTableDfs_ILF_L = [labelTable1, labelTable2, labelTable4, labelTable5]
    LabelTable_ILF_L_01 = pd.concat(LabelTableDfs_ILF_L)
    LabelTable_ILF_L_01.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_LH_ILFL_01.label', header=None, index=None, sep=' ', mode='a')
            
    LabelTableDfs_ILF_L_02 = [labelTable8, labelTable10, labelTable11, labelTable16]
    LabelTable_ILF_L_02 = pd.concat(LabelTableDfs_ILF_L_02)
    LabelTable_ILF_L_02.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_LH_ILFL_02.label', header=None, index=None, sep=' ', mode='a')
            
    #MdLF
    LabelTableDfs_MdLF_L = [labelTable1, labelTable2, labelTable3, labelTable4, labelTable5]
    LabelTable_MdLF_L_01 = pd.concat(LabelTableDfs_MdLF_L)
    LabelTable_MdLF_L_01.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_LH_MdLFL_01.label', header=None, index=None, sep=' ', mode='a')
            
    LabelTableDfs_MdLF_L_02 = [labelTable8, labelTable16]
    LabelTable_MdLF_L_02 = pd.concat(LabelTableDfs_MdLF_L_02)
    LabelTable_MdLF_L_02.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_LH_MdLFL_02.label', header=None, index=None, sep=' ', mode='a')

    #OR            
    labelTable8.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_LH_ORL_01.label', header=None, index=None, sep=' ', mode='a')

    #pARC
    LabelTableDfs_pARC_L = [labelTable12, labelTable11, labelTable5, labelTable10, labelTable6, labelTable8]
    LabelTable_pARC_L_01 = pd.concat(LabelTableDfs_pARC_L)
    LabelTable_pARC_L_01.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_LH_pARCL_01.label', header=None, index=None, sep=' ', mode='a')

    labelTable16.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_LH_pARCL_02.label', header=None, index=None, sep=' ', mode='a')
            
    #SLF
    LabelTableDfs_SLF_L = [labelTable7, labelTable15]
    LabelTable_SLF_L_01 = pd.concat(LabelTableDfs_SLF_L)
    LabelTable_SLF_L_01.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_LH_SLFL_01.label', header=None, index=None, sep=' ', mode='a')
            
    LabelTableDfs_SLF_L_02 = [labelTable12, labelTable16]
    LabelTable_SLF_L_02 = pd.concat(LabelTableDfs_SLF_L_02)
    LabelTable_SLF_L_02.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_LH_SLFL_02.label', header=None, index=None, sep=' ', mode='a')
            
    #UNC
    LabelTableDfs_UNC_L = [labelTable7, labelTable15]
    LabelTable_UNC_L_01 = pd.concat(LabelTableDfs_UNC_L)
    LabelTable_UNC_L_01.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_LH_UNCL_01.label', header=None, index=None, sep=' ', mode='a')
            
    LabelTableDfs_UNC_L_02 = [labelTable1, labelTable2]
    LabelTable_UNC_L_02 = pd.concat(LabelTableDfs_UNC_L_02)
    LabelTable_UNC_L_02.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_LH_UNCL_02.label', header=None, index=None, sep=' ', mode='a')

    #VOF
    LabelTableDfs_VOF_L = [labelTable11, labelTable10, labelTable8]
    LabelTable_VOF_L_01 = pd.concat(LabelTableDfs_VOF_L)
    LabelTable_VOF_L_01.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_LH_VOFL_01.label', header=None, index=None, sep=' ', mode='a')

    labelTable16.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_LH_VOFL_02.label', header=None, index=None, sep=' ', mode='a')

    ###############################################################################
    # Step 15: Load in labels for right hemisphere and save them in different ROIs
    ###############################################################################
         
    labelTable0 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/rh.???.label', skiprows=[0,1], sep='  ', header=None, engine='python')          
    labelTable0[4] = [x.split(' ')[0] for x in labelTable0[3]]
    labelTable0[5] = [x.split(' ')[-1] for x in labelTable0[3]]
    labelTable0 = labelTable0.drop(columns=3)
        
    labelTable1 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/rh.R_Anterior_temporal_lobe_medial.label', skiprows=[0,1], sep='  ', header=None, engine='python')
    labelTable1[4] = [x.split(' ')[0] for x in labelTable1[3]]
    labelTable1[5] = [x.split(' ')[-1] for x in labelTable1[3]]
    labelTable1 = labelTable1.drop(columns=3)
        
    labelTable2 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/rh.R_Anterior_temporal_lobe_lateral.label', skiprows=[0,1], sep='  ', header=None, engine='python')
    labelTable2[4] = [x.split(' ')[0] for x in labelTable2[3]]
    labelTable2[5] = [x.split(' ')[-1] for x in labelTable2[3]]
    labelTable2 = labelTable2.drop(columns=3)
        
    labelTable3 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/rh.R_Gyri_parahippocampalis_et_ambiens_anterior.label', skiprows=[0,1], sep='  ', header=None, engine='python')
    labelTable3[4] = [x.split(' ')[0] for x in labelTable3[3]]
    labelTable3[5] = [x.split(' ')[-1] for x in labelTable3[3]]
    labelTable3 = labelTable3.drop(columns=3)
        
    labelTable4 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/rh.R_Superior_temporal_gyrus_middle.label', skiprows=[0,1], sep='  ', header=None, engine='python')
    labelTable4[4] = [x.split(' ')[0] for x in labelTable4[3]]
    labelTable4[5] = [x.split(' ')[-1] for x in labelTable4[3]]
    labelTable4 = labelTable4.drop(columns=3)
        
    labelTable5 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/rh.R_Medial_and_inferior_temporal_gyri_anterior.label', skiprows=[0,1], sep='  ', header=None, engine='python')
    labelTable5[4] = [x.split(' ')[0] for x in labelTable5[3]]
    labelTable5[5] = [x.split(' ')[-1] for x in labelTable5[3]]
    labelTable5 = labelTable5.drop(columns=3)
        
    labelTable6 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/rh.R_Lateral_occipitotemporal_gyrus_gyrus_fusiformis_anterior.label', skiprows=[0,1], sep='  ', header=None, engine='python')
    labelTable6[4] = [x.split(' ')[0] for x in labelTable6[3]]
    labelTable6[5] = [x.split(' ')[-1] for x in labelTable6[3]]
    labelTable6 = labelTable6.drop(columns=3)
        
    labelTable7 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/rh.R_Insula.label', skiprows=[0,1], sep='  ', header=None, engine='python')
    labelTable7[4] = [x.split(' ')[0] for x in labelTable7[3]]
    labelTable7[5] = [x.split(' ')[-1] for x in labelTable7[3]]
    labelTable7 = labelTable7.drop(columns=3)
        
    labelTable8 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/rh.R_Occipital_lobe.label', skiprows=[0,1], sep='  ', header=None, engine='python')
    labelTable8[4] = [x.split(' ')[0] for x in labelTable8[3]]
    labelTable8[5] = [x.split(' ')[-1] for x in labelTable8[3]]
    labelTable8 = labelTable8.drop(columns=3)
        
    labelTable9 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/rh.R_Gyri_parahippocampalis_et_ambiens_posterior.label', skiprows=[0,1], sep='  ', header=None, engine='python')
    labelTable9[4] = [x.split(' ')[0] for x in labelTable9[3]]
    labelTable9[5] = [x.split(' ')[-1] for x in labelTable9[3]]
    labelTable9 = labelTable9.drop(columns=3)
        
    labelTable10 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/rh.R_Lateral_occipitotemporal_gyrus_gyrus_fusiformis_posterior.label', skiprows=[0,1], sep='  ', header=None, engine='python')
    labelTable10[4] = [x.split(' ')[0] for x in labelTable10[3]]
    labelTable10[5] = [x.split(' ')[-1] for x in labelTable10[3]]
    labelTable10 = labelTable10.drop(columns=3)
        
    labelTable11 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/rh.R_Medial_and_inferior_temporal_gyri_posterior.label', skiprows=[0,1], sep='  ', header=None, engine='python')
    labelTable11[4] = [x.split(' ')[0] for x in labelTable11[3]]
    labelTable11[5] = [x.split(' ')[-1] for x in labelTable11[3]]
    labelTable11 = labelTable11.drop(columns=3)
        
    labelTable12 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/rh.R_Superior_temporal_gyrus_posterior.label', skiprows=[0,1], sep='  ', header=None, engine='python')
    labelTable12[4] = [x.split(' ')[0] for x in labelTable12[3]]
    labelTable12[5] = [x.split(' ')[-1] for x in labelTable12[3]]
    labelTable12 = labelTable12.drop(columns=3)
        
    labelTable13 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/rh.R_Cingulate_gyrus_anterior.label', skiprows=[0,1], sep='  ', header=None, engine='python')
    labelTable13[4] = [x.split(' ')[0] for x in labelTable13[3]]
    labelTable13[5] = [x.split(' ')[-1] for x in labelTable13[3]]
    labelTable13 = labelTable13.drop(columns=3)
        
    labelTable14 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/rh.R_Cingulate_gyrus_posterior.label', skiprows=[0,1], sep='  ', header=None, engine='python')
    labelTable14[4] = [x.split(' ')[0] for x in labelTable14[3]]
    labelTable14[5] = [x.split(' ')[-1] for x in labelTable14[3]]
    labelTable14 = labelTable14.drop(columns=3)
       
    labelTable15 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/rh.R_Frontal_lobe.label', skiprows=[0,1], sep='  ', header=None, engine='python')
    labelTable15[4] = [x.split(' ')[0] for x in labelTable15[3]]
    labelTable15[5] = [x.split(' ')[-1] for x in labelTable15[3]]
    labelTable15 = labelTable15.drop(columns=3)
        
    labelTable16 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/annotation/rh.R_Parietal_lobe.label', skiprows=[0,1], sep='  ', header=None, engine='python')
    labelTable16[4] = [x.split(' ')[0] for x in labelTable16[3]]
    labelTable16[5] = [x.split(' ')[-1] for x in labelTable16[3]]
    labelTable16 = labelTable16.drop(columns=3)

    #ARC
    labelTable15.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_RH_ARCR_01.label', header=None, index=None, sep=' ', mode='a')
            
    LabelTableDfs_ARC_R = [labelTable2, labelTable4, labelTable5, labelTable6, labelTable10, labelTable11]
    LabelTable_ARC_R_02 = pd.concat(LabelTableDfs_ARC_R)
    LabelTable_ARC_R_02.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_RH_ARCR_02.label', header=None, index=None, sep=' ', mode='a')
            
    #ATR
    labelTable15.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_RH_ATRR_01.label', header=None, index=None, sep=' ', mode='a')
                   
    #CGC
    LabelTableDfs_CGC_R = [labelTable13, labelTable15]
    LabelTable_CGC_R_01 = pd.concat(LabelTableDfs_CGC_R)
    LabelTable_CGC_R_01.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_RH_CGCR_01.label', header=None, index=None, sep=' ', mode='a')
            
    LabelTableDfs_CGC_R_02 = [labelTable9, labelTable14, labelTable16]
    LabelTable_CGC_R_02 = pd.concat(LabelTableDfs_CGC_R_02)
    LabelTable_CGC_R_02.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_RH_CGCR_02.label', header=None, index=None, sep=' ', mode='a')
            
    #CST
    LabelTableDfs_CST_R = [labelTable12, labelTable15, labelTable16]
    LabelTable_CST_R_01 = pd.concat(LabelTableDfs_CST_R)
    LabelTable_CST_R_01.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_RH_CSTR_01.label', header=None, index=None, sep=' ', mode='a')
                        
    #FA
    labelTable15.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_RH_FA_01.label', header=None, index=None, sep=' ', mode='a')
                     
    #FP
    LabelTableDfs_FP_R = [labelTable8, labelTable16]
    LabelTable_FP_R_01 = pd.concat(LabelTableDfs_FP_R)
    LabelTable_FP_R_01.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_RH_FP_01.label', header=None, index=None, sep=' ', mode='a')
                        
    #IFO
    labelTable15.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_RH_IFOR_01.label', header=None, index=None, sep=' ', mode='a')
        
    labelTable8.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_RH_IFOR_02.label', header=None, index=None, sep=' ', mode='a')
            
    #ILF
    LabelTableDfs_ILF_R = [labelTable1, labelTable2, labelTable4, labelTable5]
    LabelTable_ILF_R_01 = pd.concat(LabelTableDfs_ILF_R)
    LabelTable_ILF_R_01.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_RH_ILFR_01.label', header=None, index=None, sep=' ', mode='a')
            
    LabelTableDfs_ILF_R_02 = [labelTable8, labelTable10, labelTable11, labelTable16]
    LabelTable_ILF_R_02 = pd.concat(LabelTableDfs_ILF_R_02)
    LabelTable_ILF_R_02.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_RH_ILFR_02.label', header=None, index=None, sep=' ', mode='a')
            
    #MdLF
    LabelTableDfs_MdLF_R = [labelTable1, labelTable2, labelTable3, labelTable4, labelTable5]
    LabelTable_MdLF_R_01 = pd.concat(LabelTableDfs_MdLF_R)
    LabelTable_MdLF_R_01.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_RH_MdLFR_01.label', header=None, index=None, sep=' ', mode='a')
            
    LabelTableDfs_MdLF_R_02 = [labelTable8, labelTable16]
    LabelTable_MdLF_R_02 = pd.concat(LabelTableDfs_MdLF_R_02)
    LabelTable_MdLF_R_02.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_RH_MdLFR_02.label', header=None, index=None, sep=' ', mode='a')

    #OR
    labelTable8.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_RH_ORR_01.label', header=None, index=None, sep=' ', mode='a')

    #pARC
    LabelTableDfs_pARC_R = [labelTable12, labelTable11, labelTable5, labelTable10, labelTable6, labelTable8]
    LabelTable_pARC_R_01 = pd.concat(LabelTableDfs_pARC_R)
    LabelTable_pARC_R_01.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_RH_pARCR_01.label', header=None, index=None, sep=' ', mode='a')

    labelTable16.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_RH_pARCR_02.label', header=None, index=None, sep=' ', mode='a')
            
    #SLF
    LabelTableDfs_SLF_R = [labelTable7, labelTable15]
    LabelTable_SLF_R_01 = pd.concat(LabelTableDfs_SLF_R)
    LabelTable_SLF_R_01.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_RH_SLFR_01.label', header=None, index=None, sep=' ', mode='a')
            
    LabelTableDfs_SLF_R_02 = [labelTable12, labelTable16]
    LabelTable_SLF_R_02 = pd.concat(LabelTableDfs_SLF_R_02)
    LabelTable_SLF_R_02.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_RH_SLFR_02.label', header=None, index=None, sep=' ', mode='a')
            
    #UNC
    LabelTableDfs_UNC_R = [labelTable7, labelTable15]
    LabelTable_UNC_R_01 = pd.concat(LabelTableDfs_UNC_R)
    LabelTable_UNC_R_01.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_RH_UNCR_01.label', header=None, index=None, sep=' ', mode='a')
            
    LabelTableDfs_UNC_R_02 = [labelTable1, labelTable2]
    LabelTable_UNC_R_02 = pd.concat(LabelTableDfs_UNC_R_02)
    LabelTable_UNC_R_02.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_RH_UNCR_02.label', header=None, index=None, sep=' ', mode='a')

    #VOF
    LabelTableDfs_VOF_R = [labelTable11, labelTable10, labelTable8]
    LabelTable_VOF_R_01 = pd.concat(LabelTableDfs_VOF_R)
    LabelTable_VOF_R_01.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_RH_VOFR_01.label', header=None, index=None, sep=' ', mode='a')

    labelTable16.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_RH_VOFR_02.label', header=None, index=None, sep=' ', mode='a')

    ###############################################################################
    # Step 16: Merge surface endpoint file and label together by vertices to get endpoint-vertices
    ###############################################################################

    leftBundles = ['ARCL', 'ATRL', 'CGCL', 'CSTL', 'FA', 'FP', 'IFOL', 'ILFL', 'MdLFL', 'ORL', 'pARCL', 'SLFL', 'UNCL', 'VOFL']
    outfolder = f"output/sub-{subject}/ses-{session}/label/FinalLabels"

    for bundle in leftBundles:
        endpoints = nib.freesurfer.mghformat.load(f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_bundle-{bundle}_lh_proj_max.mgh')
        endpointValues = endpoints.get_fdata()
        dfEndpointMap=pd.DataFrame(endpointValues[:,0])
        dfEndpointMap.columns=['endpointValues']
        dfEndpointMap['verticeNr'] = dfEndpointMap.index
        dfEndpointMap['verticeNr'] = dfEndpointMap['verticeNr'].astype(str)
            
        labelTableBundle01 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/ROIs/sub-{subject}_ses-{session}_LH_{bundle}_01.label', sep=' ', header=None)
        labelTableBundle01.columns =['verticeNr', 'x', 'y', 'z', 'value']
        labelTableBundle01['verticeNr'] = labelTableBundle01['verticeNr'].astype(str)
        labelTableMatch1=pd.merge(labelTableBundle01, dfEndpointMap, on='verticeNr')
            
        labelTableMatch1=labelTableMatch1[labelTableMatch1['endpointValues'] > 0.0001]
        labelTableMatch1=labelTableMatch1.drop(['value'], axis=1)
        labelTableMatch1=labelTableMatch1.drop_duplicates('verticeNr')
        labelTableMatch1.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_LH_{bundle}_01_matched.label', header=None, index=None, sep=' ', mode='a')

        length1 = len(labelTableMatch1.index)
        list_of_lines1 = [f'#!ascii label  , from subject sub-{subject}_ses-{session} vox2ras=TkReg', f'{length1}']
        prepend_multiple_lines(f"output/sub-{subject}/ses-{session}/label/FinalLabels/sub-{subject}_ses-{session}_LH_{bundle}_01_matched.label", list_of_lines1)
            
        try:
            labelTableBundle02 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/ROIs/sub-{subject}_ses-{session}_LH_{bundle}_02.label', sep=' ', header=None)
            labelTableBundle02.columns =['verticeNr', 'x', 'y', 'z', 'value']
            labelTableBundle02['verticeNr'] = labelTableBundle02['verticeNr'].astype(str)
                                           
            labelTableMatch2=pd.merge(labelTableBundle02, dfEndpointMap, on='verticeNr')
            labelTableMatch2=labelTableMatch2[labelTableMatch2['endpointValues'] > 0.0001]
            labelTableMatch2=labelTableMatch2.drop(['value'], axis=1)
            labelTableMatch2=labelTableMatch2.drop_duplicates('verticeNr')
            labelTableMatch2.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_LH_{bundle}_02_matched.label', header=None, index=None, sep=' ', mode='a')
            
            length2 = len(labelTableMatch2.index)
            
            list_of_lines2 = [f'#!ascii label  , from subject sub-{subject}_ses-{session} vox2ras=TkReg', f'{length2}']
            prepend_multiple_lines(f"output/sub-{subject}/ses-{session}/label/FinalLabels/sub-{subject}_ses-{session}_LH_{bundle}_02_matched.label", list_of_lines2)
        except:
            pass



    rightBundles = ['ARCR', 'ATRR', 'CGCR', 'CSTR', 'FA', 'FP', 'IFOR', 'ILFR', 'MdLFR', 'ORR', 'pARCR', 'SLFR', 'UNCR', 'VOFR']

    for bundle in rightBundles:
        endpoints = nib.freesurfer.mghformat.load(f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_bundle-{bundle}_rh_proj_max.mgh')
        endpointValues = endpoints.get_fdata()
        dfEndpointMap=pd.DataFrame(endpointValues[:,0])
        dfEndpointMap.columns=['endpointValues']
        dfEndpointMap['verticeNr'] = dfEndpointMap.index
        dfEndpointMap['verticeNr'] = dfEndpointMap['verticeNr'].astype(str)
            
        labelTableBundle01 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/ROIs/sub-{subject}_ses-{session}_RH_{bundle}_01.label', sep=' ', header=None)
        labelTableBundle01.columns =['verticeNr', 'x', 'y', 'z', 'value']
        labelTableBundle01['verticeNr'] = labelTableBundle01['verticeNr'].astype(str)
            
        labelTableMatch1=pd.merge(labelTableBundle01, dfEndpointMap, on='verticeNr')
        labelTableMatch1=labelTableMatch1[labelTableMatch1['endpointValues'] > 0.0001]
        labelTableMatch1=labelTableMatch1.drop(['value'], axis=1)
        labelTableMatch1=labelTableMatch1.drop_duplicates('verticeNr')
        labelTableMatch1.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_RH_{bundle}_01_matched.label', header=None, index=None, sep=' ', mode='a')
            
        length1 = len(labelTableMatch1.index)
        list_of_lines1 = [f'#!ascii label  , from subject sub-{subject}_ses-{session} vox2ras=TkReg', f'{length1}']
        prepend_multiple_lines(f"output/sub-{subject}/ses-{session}/label/FinalLabels/sub-{subject}_ses-{session}_RH_{bundle}_01_matched.label", list_of_lines1)
            
        try:
            labelTableBundle02 = pd.read_table(f'output/sub-{subject}/ses-{session}/label/ROIs/sub-{subject}_ses-{session}_RH_{bundle}_02.label', sep=' ', header=None)
            labelTableBundle02.columns =['verticeNr', 'x', 'y', 'z', 'value']
            labelTableBundle02['verticeNr'] = labelTableBundle02['verticeNr'].astype(str)
                       
            labelTableMatch2=pd.merge(labelTableBundle02, dfEndpointMap, on='verticeNr')
            labelTableMatch2=labelTableMatch2[labelTableMatch2['endpointValues'] > 0.0001]
            labelTableMatch2=labelTableMatch2.drop(['value'], axis=1)
            labelTableMatch2=labelTableMatch2.drop_duplicates('verticeNr')
            labelTableMatch2.to_csv(f'{outfolder}/sub-{subject}_ses-{session}_RH_{bundle}_02_matched.label', header=None, index=None, sep=' ', mode='a')
            
            length2 = len(labelTableMatch2.index)
            list_of_lines2 = [f'#!ascii label  , from subject sub-{subject}_ses-{session} vox2ras=TkReg', f'{length2}']
            prepend_multiple_lines(f"output/sub-{subject}/ses-{session}/label/FinalLabels/sub-{subject}_ses-{session}_RH_{bundle}_02_matched.label", list_of_lines2)
        except:
            pass

     ###############################################################################
    # Step 17. Convert surface myelinmap.gii files to .mgh files
    ###############################################################################


    converttomgh_LH = subprocess.run(
        [
            'mri_convert',
                f'input/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_hemi-left_myelinmap.shape.gii',
                f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_hemi-L_space-T2w_myelinmap.shape.mgh'
        ],
        check=True,
        capture_output=True,
        text=True
        )
    print_quiet(converttomgh_LH.stdout)
    print_quiet(converttomgh_LH.stdout)


    converttomgh_RH = subprocess.run(
        [
            'mri_convert',
                f'input/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_hemi-right_myelinmap.shape.gii',
                f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_hemi-R_space-T2w_myelinmap.shape.mgh'
        ],
        check=True,
        capture_output=True,
        text=True
        )
    print_quiet(converttomgh_RH.stdout)
    print_quiet(converttomgh_RH.stdout)

    ###############################################################################
    # Step 18: Find myelinvalue in vertices of endpoints (left hemisphere)
    ###############################################################################

    for bundle in leftBundles:
        t1wt2wMap_LH = nib.freesurfer.mghformat.load(f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_hemi-L_space-T2w_myelinmap.shape.mgh')
        t1wt2wValues_LH = t1wt2wMap_LH.get_fdata()
        t1wt2wValuesMap_LH=pd.DataFrame(t1wt2wValues_LH[:,0])
        t1wt2wValuesMap_LH.columns=['t1wt2wValues_LH']
        t1wt2wValuesMap_LH['verticeNr'] = t1wt2wValuesMap_LH.index
        t1wt2wValuesMap_LH['verticeNr'] = t1wt2wValuesMap_LH['verticeNr'].astype(str)
        t1wt2wValuesMap_LH

        rois1_LH = pd.read_table(f"output/sub-{subject}/ses-{session}/label/FinalLabels/sub-{subject}_ses-{session}_LH_{bundle}_01_matched.label",  skiprows=[0,1], sep=' ', header=None)
        rois1_LH.columns =['verticeNr', 'x', 'y', 'z', 'endpointdensity']
        rois1_LH['verticeNr'] = rois1_LH['verticeNr'].astype(str)
        rois1_LH
        t1wt2w_data_rois1_LH=pd.merge(t1wt2wValuesMap_LH, rois1_LH, on='verticeNr')
        t1wt2w_data_rois1_LH["subjectID"]=f'{subject}'
        t1wt2w_data_rois1_LH["sessionID"]=f'{session}'
        t1wt2w_data_rois1_LH["tractID"]=f'{bundle}'
        t1wt2w_data_rois1_LH["ROI"]='1'
        t1wt2w_data_rois1_LH["hemisphere"]='L'
        t1wt2w_data_rois1_LH

        try:
            rois2_LH = pd.read_table(f"output/sub-{subject}/ses-{session}/label/FinalLabels/sub-{subject}_ses-{session}_LH_{bundle}_02_matched.label",  skiprows=[0,1], sep=' ', header=None)
            rois2_LH.columns =['verticeNr', 'x', 'y', 'z', 'endpointdensity']
            rois2_LH['verticeNr'] = rois2_LH['verticeNr'].astype(str)
            rois2_LH
            t1wt2w_data_rois2_LH=pd.merge(t1wt2wValuesMap_LH, rois2_LH, on='verticeNr')
            t1wt2w_data_rois2_LH["subjectID"]=f'{subject}'
            t1wt2w_data_rois2_LH["sessionID"]=f'{session}'
            t1wt2w_data_rois2_LH["tractID"]=f'{bundle}'
            t1wt2w_data_rois2_LH["ROI"]='2'
            t1wt2w_data_rois2_LH["hemisphere"]='L'
            t1wt2w_data_rois2_LH

            t1wt2wdataLH = pd.concat([t1wt2w_data_rois1_LH, t1wt2w_data_rois2_LH], ignore_index=True)
            t1wt2wdataLH
            t1wt2wdataLH.to_csv(f'output/sub-{subject}/ses-{session}/t1t2values/GreyMatterT1T2_{bundle}_LH.csv', sep=' ', mode='a')
            
        except:
            t1wt2w_data_rois1_LH.to_csv(f'output/sub-{subject}/ses-{session}/t1t2values/GreyMatterT1T2_{bundle}_LH.csv', sep=' ', mode='a')

    ###############################################################################
    # Step 19: Find myelinvalue in vertices of endpoints (right hemisphere)
    ###############################################################################

    for bundle in rightBundles:
        t1wt2wMap_RH = nib.freesurfer.mghformat.load(f'output/sub-{subject}/ses-{session}/surf/sub-{subject}_ses-{session}_hemi-R_space-T2w_myelinmap.shape.mgh')
        t1wt2wValues_RH = t1wt2wMap_RH.get_fdata()
        t1wt2wValuesMap_RH=pd.DataFrame(t1wt2wValues_RH[:,0])
        t1wt2wValuesMap_RH.columns=['t1wt2wValues_RH']
        t1wt2wValuesMap_RH['verticeNr'] = t1wt2wValuesMap_RH.index
        t1wt2wValuesMap_RH['verticeNr'] = t1wt2wValuesMap_RH['verticeNr'].astype(str)
        t1wt2wValuesMap_RH

        rois1_RH = pd.read_table(f"output/sub-{subject}/ses-{session}/label/FinalLabels/sub-{subject}_ses-{session}_RH_{bundle}_01_matched.label",  skiprows=[0,1], sep=' ', header=None)
        rois1_RH.columns =['verticeNr', 'x', 'y', 'z', 'endpointdensity']
        rois1_RH['verticeNr'] = rois1_RH['verticeNr'].astype(str)
        rois1_RH
        t1wt2w_data_rois1_RH=pd.merge(t1wt2wValuesMap_RH, rois1_RH, on='verticeNr')
        t1wt2w_data_rois1_RH["subjectID"]=f'{subject}'
        t1wt2w_data_rois1_RH["sessionID"]=f'{session}'
        t1wt2w_data_rois1_RH["tractID"]=f'{bundle}'
        t1wt2w_data_rois1_RH["ROI"]='1'
        t1wt2w_data_rois1_RH["hemisphere"]='R'
        t1wt2w_data_rois1_RH

        try:
            rois2_RH = pd.read_table(f"output/sub-{subject}/ses-{session}/label/FinalLabels/sub-{subject}_ses-{session}_RH_{bundle}_02_matched.label",  skiprows=[0,1], sep=' ', header=None)
            rois2_RH.columns =['verticeNr', 'x', 'y', 'z', 'endpointdensity']
            rois2_RH['verticeNr'] = rois2_LH['verticeNr'].astype(str)
            rois2_RH
            t1wt2w_data_rois2_RH=pd.merge(t1wt2wValuesMap_RH, rois2_RH, on='verticeNr')
            t1wt2w_data_rois2_RH["subjectID"]=f'{subject}'
            t1wt2w_data_rois2_RH["sessionID"]=f'{session}'
            t1wt2w_data_rois2_RH["tractID"]=f'{bundle}'
            t1wt2w_data_rois2_RH["ROI"]='2'
            t1wt2w_data_rois2_RH["hemisphere"]='R'
            t1wt2w_data_rois2_RH
    
            t1wt2wdataRH = pd.concat([t1wt2w_data_rois1_RH, t1wt2w_data_rois2_RH], ignore_index=True)
            t1wt2wdataRH
            t1wt2wdataRH.to_csv(f'output/sub-{subject}/ses-{session}/t1t2values/GreyMatterT1T2_{bundle}_RH.csv', sep=' ', mode='a')
            
        except:
            t1wt2w_data_rois1_RH.to_csv(f'output/sub-{subject}/ses-{session}/t1t2values/GreyMatterT1T2_{bundle}_RH.csv', sep=' ', mode='a')
